#<font color=red>Before you use the simulator, press **Ctrl-F9** to execute all code on the sheet</font>



#Introduction

HuckIt is a rigid body dynamics solver that predicts the flight path of a flying disc based a given set of throw parameters. It uses lift and drag coefficient data collected from wind tunnel measurements to calculate the forces on a disc in flight. The body forces are then used to calculate the changes in velocity, position and roll angle of the disc. After executing the code on the sheet, you can skip down to the User Interface to experiment with throw parameters and see the effects on the flight path.

# Physics Simulator Code

##References

The simulator is based on the method used in Simulation of a spin stabilized sports disc, Crowther and Potts (2007)
A slightly different coordinate system is used in this model so some of the sign conventions may not match up.
The formulation assumes the following
 - Spin rate is constant and does not decay throughout the flight
 - Aerodynamic side force on the disc is negligible
 - Aerodynamic rolling moment is negligible (center of pressure is centered left to right on the disc)
 - Disc is thrown without wobble

Four coordinate systems are used in the model;
- Ground: From the thrower's position, positive $x$ is forward, positive $y$ is to the left, positive $z$ is up. The position and orientation of the disc is calculated in this coordinate system.
- Disc: The coordinate system that moves with the disc. The $x$-axis runs from tail to nose of the disc, $y$-axis from right to left, and $z$-axis is normal to the flight plate pointing up.
- Zero side-slip: A coordinate system that is used to transition to the wind coordinates. The only difference between this and the disc coordinate system is a rotation about the $z$-axis by the side-slip angle, beta. The rolling rate is calculated in this coordinate system.
- Wind: This coordinate system points directly into the oncoming airflow, and is meant to replicate the coordinate system in a wind tunnel. The only difference between this and the zero side-slip coordinate system is a rotation about the $y$-axis by the angle of attack, alpha. The aerodynamic forces (lift, drag, and pitching moment) and body accelerations are calculated in this coordinate system.


## Setup

In [2]:
import numpy as np

## Define Coordinate Transform Functions

In [3]:
def T_gd(angles):
  phi = angles[0]
  theta = angles[1]
  psi = angles[2]
  return np.array([[np.cos(theta)*np.cos(psi), np.sin(phi)*np.sin(theta)*np.cos(psi) - np.cos(phi)*np.sin(psi), np.cos(phi)*np.sin(theta)*np.cos(psi) + np.sin(phi)*np.sin(psi)],
                   [np.cos(theta)*np.sin(psi), np.sin(phi)*np.sin(theta)*np.sin(psi) + np.cos(phi)*np.cos(psi), np.cos(phi)*np.sin(theta)*np.sin(psi) - np.sin(phi)*np.cos(psi)],
                   [-np.sin(theta),            np.sin(phi)*np.cos(theta),                                       np.cos(phi)*np.cos(theta)                                      ]])

def T_dg(angles):
  phi = angles[0]
  theta = angles[1]
  psi = angles[2]
  return np.array([[np.cos(theta)*np.cos(psi),                                      np.cos(theta)*np.sin(psi),                                      -np.sin(theta)            ],
                    [np.sin(phi)*np.sin(theta)*np.cos(psi) - np.cos(phi)*np.sin(psi), np.sin(phi)*np.sin(theta)*np.sin(psi) + np.cos(phi)*np.cos(psi), np.sin(phi)*np.cos(theta)],
                    [np.cos(phi)*np.sin(theta)*np.cos(psi) + np.sin(phi)*np.sin(psi), np.cos(phi)*np.sin(theta)*np.sin(psi) - np.sin(phi)*np.cos(psi), np.cos(phi)*np.cos(theta)]])

def T_ds(beta):
  return np.array([[np.cos(beta), -np.sin(beta), 0],
                   [np.sin(beta),  np.cos(beta), 0],
                   [0,             0,            1]])

def T_sd(beta):
  return np.array([[np.cos(beta),  np.sin(beta), 0],
                   [-np.sin(beta), np.cos(beta), 0],
                   [0,             0,            1]])

def T_sw(alpha):
  return np.array([[np.cos(alpha), 0, -np.sin(alpha)],
                   [0,             1, 0            ],
                   [np.sin(alpha), 0, np.cos(alpha)]])

def T_ws(alpha):
  return np.array([[np.cos(alpha), 0, np.sin(alpha)],
                   [0,             1, 0            ],
                   [-np.sin(alpha), 0, np.cos(alpha)]])

## Define Discs

In [4]:
'''
Define Disc class
Property definitions
- name: Name of disc as listed by the manufacturer
- aoarange: Angle of Attack at which the aerodynamic coefficients have been measured in rad. Must be sorted in ascending order.
- cl: Coefficient of lift at various AoA
- cd: Coefficient of drag at various AoA
- cm: Coefficient of pitching moment at various AoA
- jxy: Normalized mass moment of inertia about the roll/pitch axis in m^2. Multiply by mass to get true MMOI.
- jz: Normalized mass moment of inertia about the spin axis in m^2. Multiply by mass to get true MMOI.
- diam: Disc diameter in m
- mass: Mass of disc in kg

Method definitions
- getCl: Returns the coefficient of lift at the specified Angle of Attack
- getCd: Returns the coefficient of drag at the specified Angle of Attack
- getCm: Returns the coefficient of moment at the specified Angle of Attack
'''

discList = [(0,0)]

class Disc:
  def __init__(self, name):
    self.name = name
    self.cl = [0]
    self.cd = [0]
    self.cm = [0]
    self.jxy = 0
    self.jz = 0
    self.diam = 0

  def getCl(self, aoa):
    return np.interp(aoa, self.aoarange, self.cl)
  def getCd(self, aoa):
    return np.interp(aoa, self.aoarange, self.cd)
  def getCm(self, aoa):
    return np.interp(aoa, self.aoarange, self.cm)

'''
Define discs
Aerodynamic coefficients for frisbee sourced from Frisbee Aerodynamics, Potts and Crowther (2002)
'''
frisbee = Disc("Frisbee")
frisbee.diam = 0.274
frisbee.jxy = 7.69e-3
frisbee.jz = 1.01e-2
frisbee.aoarange = np.array([-1.745329252,-1.658062789,-1.570796327,-1.483529864,-1.396263402,-1.308996939,-1.221730476,-1.134464014,-1.047197551,-0.959931089,-0.174532925,-0.157079633,-0.13962634,-0.122173048,-0.104719755,-0.087266463,-0.06981317,-0.052359878,-0.034906585,-0.017453293,0,0.017453293,0.034906585,0.052359878,0.06981317,0.087266463,0.104719755,0.122173048,0.13962634,0.157079633,0.174532925,0.191986218,0.20943951,0.226892803,0.244346095,0.261799388,0.27925268,0.296705973,0.314159265,0.331612558,0.34906585,0.366519143,0.383972435,0.401425728,0.41887902,0.436332313,0.453785606,0.471238898,0.488692191,0.506145483,0.523598776,0.541052068,0.558505361,0.575958653,0.593411946,0.610865238,0.628318531,0.645771823,0.663225116,0.680678408,0.698131701,0.715584993,0.733038286,0.750491578,0.767944871,0.785398163,0.802851456,0.820304748,0.837758041,0.855211333,0.872664626,0.959931089,1.047197551,1.134464014,1.221730476,1.308996939,1.396263402,1.483529864,1.570796327,1.658062789,1.745329252])
frisbee.cl = np.array([0.15942029,0.096618357,0.009661836,-0.077294686,-0.144927536,-0.217391304,-0.299516908,-0.357487923,-0.434782609,-0.492753623,-0.234509466,-0.204388985,-0.148450947,-0.126936317,-0.096815835,-0.083907057,-0.058089501,-0.027969019,0.023666093,0.075301205,0.118330465,0.182874355,0.238812392,0.303356282,0.376506024,0.432444062,0.496987952,0.55292599,0.613166954,0.673407917,0.729345955,0.776678141,0.836919105,0.875645439,0.922977625,0.983218589,1.021944923,1.07788296,1.129518072,1.172547332,1.219879518,1.275817556,1.318846816,1.396299484,1.44363167,1.486660929,1.512478485,1.589931153,1.620051635,1.667383821,1.688898451,1.710413081,1.740533563,1.762048193,1.813683305,1.850258176,1.886833046,1.929862306,1.972891566,2.007314974,2.063253012,2.093373494,2.106282272,2.136402754,2.151462995,2.149311532,1.146729776,1.133820998,1.133820998,1.09939759,1.082185886,1.019323671,0.8647343,0.724637681,0.589371981,0.45410628,0.299516908,0.140096618,0.004830918,-0.140096618,-0.280193237])
frisbee.cd = np.array([0.81906226,0.843658724,0.848578017,0.828900846,0.811683321,0.806764028,0.789546503,0.760030746,0.748962337,0.710837817,0.162962963,0.145679012,0.12345679,0.10617284,0.10617284,0.096296296,0.088888889,0.088888889,0.086419753,0.088888889,0.09382716,0.101234568,0.113580247,0.125925926,0.133333333,0.151851852,0.166666667,0.190123457,0.216049383,0.239506173,0.264197531,0.286419753,0.309876543,0.341975309,0.367901235,0.397530864,0.427160494,0.459259259,0.498765432,0.530864198,0.572839506,0.602469136,0.641975309,0.681481481,0.72345679,0.754320988,0.787654321,0.832098765,0.864197531,0.90617284,0.920987654,0.95308642,0.975308642,1.002469136,1.032098765,1.071604938,1.103703704,1.135802469,1.182716049,1.232098765,1.295061728,1.340740741,1.380246914,1.437037037,1.491358025,1.530864198,1.009876543,1.017283951,1.039506173,1.04691358,1.064197531,1.097002306,1.146195234,1.185549577,1.212605688,1.247040738,1.269177556,1.269177556,1.29377402,1.286395081,1.276556495])
frisbee.cm = np.array([0.031216649,0.013607257,0.000266809,-0.01547492,-0.031216649,-0.046691569,-0.060565635,-0.073372465,-0.085645678,-0.095517609,-0.038247863,-0.036538462,-0.030982906,-0.027564103,-0.023504274,-0.021581197,-0.017307692,-0.014102564,-0.010042735,-0.008333333,-0.006837607,-0.008119658,-0.00982906,-0.006837607,-0.006623932,-0.004059829,-0.002350427,-0.001068376,-0.001068376,0.000641026,0.003205128,0.007478632,0.010683761,0.013461538,0.016452991,0.021153846,0.025854701,0.031410256,0.034401709,0.04017094,0.043376068,0.048931624,0.053632479,0.061111111,0.068589744,0.07542735,0.083119658,0.088888889,0.096367521,0.103846154,0.111324786,0.115811966,0.125854701,0.137820513,0.144017094,0.152564103,0.160042735,0.171794872,0.180769231,0.18974359,0.2,0.208760684,0.216239316,0.223931624,0.227991453,0.22542735,0.019871795,0.018376068,0.018162393,0.018162393,0.016452991,0.016275347,0.017609392,0.021611526,0.021611526,0.020010672,0.015741729,0.008271078,0.002401281,-0.007203842,-0.011472785])
discList[0] = (frisbee.name, frisbee)

'''
Aerodynamic coefficients for aviar, roc and wraith sourced from Dynamics and Performance of Flying Discs, Kamaruddin (2011)
The curves have been extrapolated to cover a wider range of AoA. Extreme or unusual flight paths may not be as accurate
Normalized MMOI sourced from 3D CAD models of similar discs
'''
aviar = Disc("Aviar")
aviar.aoarange =  np.array([-1.570796327,-0.5235987756,-0.0872664626,-0.06981317008,-0.05235987756,-0.03490658504,-0.01745329252,0,0.01745329252,0.03490658504,0.05235987756,0.06981317008,0.0872664626,0.1047197551,0.1221730476,0.1396263402,0.1570796327,0.1745329252,0.1919862177,0.2094395102,0.2268928028,0.2443460953,0.2617993878,0.7853981634,0.872664626,1.570796327])
aviar.cl = np.array([0,-1,-0.088,-0.049,-0.009,0.034,0.093,0.154,0.21,0.256,0.304,0.343,0.383,0.426,0.468,0.508,0.549,0.591,0.631,0.672,0.702,0.74,0.78,1.6,0.8,0])
aviar.cd = np.array([0.4,0.188,0.076,0.071,0.07,0.072,0.072,0.084,0.088,0.085,0.102,0.117,0.133,0.141,0.157,0.174,0.189,0.203,0.216,0.226,0.245,0.266,0.281,0.7,0.5,0.6])
aviar.cm = np.array([0,-0.08,-0.015,-0.016,-0.011,-0.01,-0.013,-0.018,-0.018,-0.017,-0.014,-0.014,-0.011,-0.008,-0.005,0,0.005,0.009,0.011,0.02,0.024,0.032,0.039,0.23,0.02,0])
aviar.jxy = 4.23e-3
aviar.jz = 8.46e-3
aviar.diam = 0.21
discList.append((aviar.name, aviar))

roc = Disc("Roc")
roc.aoarange = np.array([-1.570796327,-0.5235987756,-0.0872664626,-0.06981317008,-0.05235987756,-0.03490658504,-0.01745329252,0,0.01745329252,0.03490658504,0.05235987756,0.06981317008,0.0872664626,0.1047197551,0.1221730476,0.1396263402,0.1570796327,0.1745329252,0.1919862177,0.2094395102,0.2268928028,0.2443460953,0.2617993878,0.7853981634,0.872664626,1.570796327])
roc.cl = np.array([0,-0.5,-0.121,-0.088,-0.061,-0.017,0.014,0.053,0.091,0.142,0.182,0.235,0.285,0.336,0.38,0.423,0.464,0.509,0.551,0.59,0.637,0.68,0.724,1.5,0.75,0])
roc.cd = np.array([0.4,0.25,0.065,0.054,0.054,0.058,0.063,0.067,0.076,0.076,0.086,0.095,0.105,0.109,0.124,0.13,0.143,0.159,0.17,0.184,0.198,0.215,0.233,0.7,0.5,0.6])
roc.cm = np.array([0,-0.1,-0.028,-0.024,-0.023,-0.018,-0.016,-0.015,-0.013,-0.013,-0.012,-0.011,-0.007,-0.003,-0.002,0.002,0.007,0.013,0.018,0.026,0.029,0.035,0.041,0.2,0.02,0])
roc.jxy = 4.06e-3
roc.jz = 8.15e-3
roc.diam = 0.21
discList.append((roc.name, roc))

wraith = Disc("Wraith")
wraith.aoarange = np.array([-1.570796327,-0.5235987756,-0.0872664626,-0.06981317008,-0.05235987756,-0.03490658504,-0.01745329252,0,0.01745329252,0.03490658504,0.05235987756,0.06981317008,0.0872664626,0.1047197551,0.1221730476,0.1396263402,0.1570796327,0.1745329252,0.1919862177,0.2094395102,0.2268928028,0.2443460953,0.2617993878,0.7853981634,0.872664626,1.570796327])
wraith.cl = np.array([0,-0.5,-0.034,0.007,0.045,0.076,0.108,0.15,0.179,0.212,0.25,0.286,0.323,0.357,0.405,0.463,0.507,0.547,0.591,0.637,0.68,0.73,0.775,2,1,0])
wraith.cd = np.array([0.2,0.2,0.057,0.051,0.053,0.051,0.052,0.056,0.06,0.062,0.073,0.075,0.084,0.089,0.102,0.118,0.127,0.141,0.153,0.167,0.185,0.199,0.22,0.7,0.5,0.6])
wraith.cm = np.array([0,-0.15,-0.059,-0.048,-0.04,-0.035,-0.026,-0.02,-0.017,-0.01,-0.004,0.001,0.009,0.018,0.02,0.024,0.031,0.038,0.05,0.057,0.064,0.071,0.081,0.24,0.02,0])
wraith.jxy = 3.86e-3
wraith.jz = 7.70e-3
wraith.diam = 0.21
discList.append((wraith.name, wraith))

## Define Throw Class

In [5]:
'''Define Throw class'''
class Throw:
  def __init__(self, name):
    self.name = name
    self.cl = [0]
    self.cd = [0]
    self.cm = [0]

  def getDistance(self):
    distance = (self.pos_g[-1,0]**2 + self.pos_g[-1,1]**2)**0.5
    return distance


## Simulation Function

In [6]:
def huckit(disc, throw):
  '''
  huckit receives a throw object and a disc object
  '''

  '''Simulation controls'''
  dt = 0.01 # Time step length in s
  step = 0 # Time step number
  maxSteps = 1000 # Maximum number of steps allowed for simulation
  vectorArraySize = [maxSteps+1, 3]
  scalarArraySize = maxSteps+1
  t = np.zeros(scalarArraySize) # Time in s

  '''Ground coordinate system'''
  pos_g = np.zeros(vectorArraySize) # Disc position in m
  vel_g = np.zeros(vectorArraySize) # Disc velocity in m/s
  acl_g = np.zeros(vectorArraySize) # Disc acceleration in m/s^2
  ori_g = np.zeros(vectorArraySize) # Disc roll, pitch, yaw angle in rad
  rot_g = np.zeros(vectorArraySize) # Disc roll, pitch, yaw rate in rad/s

  '''Disc coordinate system'''
  acl_d = np.zeros(vectorArraySize)
  vel_d = np.zeros(vectorArraySize)
  rot_d = np.zeros(vectorArraySize)

  '''Side-slip coordinate system'''
  acl_s = np.zeros(vectorArraySize)
  vel_s = np.zeros(vectorArraySize)
  rot_s = np.zeros(vectorArraySize)
  beta = np.zeros(scalarArraySize)

  '''Wind coordinate system'''
  acl_w = np.zeros(vectorArraySize)
  vel_w = np.zeros(vectorArraySize)
  alpha = np.zeros(scalarArraySize)

  '''Aerodynamic forces'''
  drag = np.zeros(scalarArraySize)
  lift = np.zeros(scalarArraySize)
  mom = np.zeros(scalarArraySize)

  '''Define disc orientation and velocity from inputs'''
  ori_g[step] = np.array([throw.roll_angle, throw.nose_angle, 0])
  vel_g[step] = np.array([throw.speed*np.cos(throw.launch_angle), 0, throw.speed*np.sin(throw.launch_angle)])
  launch_angle_d = np.matmul(T_gd(ori_g[step]), [0, throw.launch_angle, 0])
  ori_g[step] += launch_angle_d

  '''Define environmental constants'''
  rho = 1.18 # Air density in kg/m^3
  g = 9.81 # Gravitational acceleration in m/s^2
  throw.launch_height = 1.5 # Initial height of throw in m
  pos_g[step] = np.array([[0, 0, throw.launch_height]])

  '''Define derived constants'''
  mass = disc.mass
  diam = disc.diam # Diameter of disc in m
  ixy = disc.jxy*mass # Rotational moment of inertia of disc about roll axis in kg-m^2
  iz = disc.jz*mass # Rotational moment of inertia of disc about spin axis in kg-m^2
  area = np.pi*(0.5*diam)**2 # Planform area of disc in m^2
  omega = throw.spin*throw.spindir # Assign spin direction
  weight = g*mass # Gravitational force acting on the disc center of mass in N

  '''Loop until disc hits the ground, z-position=0'''
  while pos_g[step][2] > 0:
    if step >= maxSteps: # Safety valve in case the disc never returns to earth
      break

    ii=0
    while 1:
      '''Transform ground velocity to wind coordinate system'''
      vel_d[step] = np.matmul(T_gd(ori_g[step]), vel_g[step]) # Transform ground velocity to disc coordinate system
      beta[step] = -np.arctan2(vel_d[step][1], vel_d[step][0]) # Calculate side slip angle
      vel_s[step] = np.matmul(T_ds(beta[step]), vel_d[step]) # Transform velocity to zero side-slip coordinate system
      alpha[step] = -np.arctan2(vel_s[step][2], vel_s[step][0]) # Calculate the angle of attack
      vel_w[step] = np.matmul(T_sw(alpha[step]), vel_s[step]) # Transform velocity to wind coordinate system where aerodynamic calculations can be made

      '''Transform gravity loads to wind coordinate system'''
      grav_d = np.matmul(T_gd(ori_g[step]), [0, 0, -weight])
      grav_s = np.matmul(T_ds(beta[step]), grav_d)
      grav_w = np.matmul(T_sw(alpha[step]), grav_s)

      '''Calculate aerodynamic forces on the disc'''
      drag[step] = 0.5*rho*(vel_w[step][0]**2)*area*disc.getCd(alpha[step]) # Calculate drag force in N
      lift[step] = 0.5*rho*vel_w[step][0]**2*area*disc.getCl(alpha[step]) # Calculate lift force in N
      mom[step] = 0.5*rho*vel_w[step][0]**2*area*diam*disc.getCm(alpha[step]) # Calculate pitching moment in N-m

      '''Calculate body accelerations from second law and force balances'''
      acl_w[step,0] = (-drag[step] + grav_w[0]) / mass # Calculate deceleration due to drag
      acl_w[step,2] = (lift[step] + grav_w[2]) / mass # Calculate acceleration due to lift
      acl_w[step,1] = grav_w[1] / mass # Calculate acceleration due to side loading (just gravity)
      rot_s[step,0] = -mom[step]/(omega*(ixy - iz)) # Calculate roll rate from pitching moment

      '''Tranform disc acceleration to ground coordinate system'''
      acl_s[step] = np.matmul(T_ws(alpha[step]), acl_w[step])
      acl_d[step] = np.matmul(T_sd(beta[step]), acl_s[step])
      acl_g[step] = np.matmul(T_dg(ori_g[step]), acl_d[step])

      '''Transform roll rate from zero side-slip to ground coordinate system'''
      rot_d[step] = np.matmul(T_sd(beta[step]), rot_s[step])
      rot_g[step] = np.matmul(T_dg(ori_g[step]), rot_d[step])

      '''Perform one inner iteration to refine speed and position vectors'''
      if step==0: # Do not run inner iterations for initial time step
        break
      if ii>=1: # Only run one inner iteration
        break

      '''Calculate average accelerations and rotation rates between current and previous time steps'''
      avg_acl_g = (acl_g[step-1] + acl_g[step])/2
      avg_rot_g = (rot_g[step-1] + rot_g[step])/2

      '''Calculate new velocity, position and orientation for current time step'''
      vel_g[step] = vel_g[step-1] + avg_acl_g*dt
      pos_g[step] = pos_g[step-1] + vel_g[step-1]*dt + 0.5*avg_acl_g*dt**2
      ori_g[step] = ori_g[step-1] + avg_rot_g*dt

      ii+=1

    '''Estimate disc velocity, position, and orientation at next time step'''
    vel_g[step+1] = vel_g[step] + acl_g[step]*dt
    pos_g[step+1] = pos_g[step] + vel_g[step]*dt + 0.5*acl_g[step]*dt**2
    ori_g[step+1] = ori_g[step] + rot_g[step]*dt

    '''Update simulation variables'''
    t[step+1] = t[step] + dt
    step += 1

  '''Remove unused steps from simulation data arrays and assign them to throw object'''
  throw.t = np.resize(t,step)
  throw.lift = np.resize(lift,step)
  throw.drag = np.resize(drag,step)
  throw.mom = np.resize(mom,step)
  throw.alpha = np.resize(alpha,step)
  throw.beta = np.resize(beta,step)
  throw.pos_g = np.resize(pos_g,[step,3])
  throw.vel_g = np.resize(vel_g,[step,3])
  throw.acl_g = np.resize(acl_g,[step,3])
  throw.ori_g = np.resize(ori_g,[step,3])
  throw.rot_g = np.resize(rot_g,[step,3])
  throw.vel_d = np.resize(vel_d,[step,3])
  throw.vel_w = np.resize(vel_w,[step,3])
  throw.acl_w = np.resize(acl_w,[step,3])
  throw.rot_s = np.resize(rot_s,[step,3])

  return throw

# Interface Code

## Libraries

In [7]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import interactive
from IPython.display import display,clear_output
%matplotlib inline

'''Prevents display of extra figure'''
%config InlineBackend.close_figures=False
plt.ioff()

'''Tools for 3D plotting'''
from mpl_toolkits.mplot3d.proj3d import proj_transform
from matplotlib.text import Annotation
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
!pip install meshio[all]
import meshio

class Arrow3D(FancyArrowPatch):
    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0,0), (0,0), *args, **kwargs)
        self._xyz = (x,y,z)
        self._dxdydz = (dx,dy,dz)

    def draw(self, renderer):
        x1,y1,z1 = self._xyz
        dx,dy,dz = self._dxdydz
        x2,y2,z2 = (x1+dx,y1+dy,z1+dz)

        xs, ys, zs = proj_transform((x1,x2),(y1,y2),(z1,z2), renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        super().draw(renderer)

def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)

setattr(Axes3D,'arrow3D',_arrow3D)

class Annotation3D(Annotation):
    def __init__(self, text, xyz, *args, **kwargs):
        super().__init__(text, xy=(0,0), *args, **kwargs)
        self._xyz = xyz

    def draw(self, renderer):
        x2, y2, z2 = proj_transform(*self._xyz, renderer.M)
        self.xy=(x2,y2)
        super().draw(renderer)

def _annotate3D(ax,text, xyz, *args, **kwargs):
    '''Add anotation `text` to an `Axes3d` instance.'''

    annotation= Annotation3D(text, xyz, *args, **kwargs)
    ax.add_artist(annotation)

setattr(Axes3D,'annotate3D',_annotate3D)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.7/167.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.6 MB/s eta 0:00:00


## Define Flight Path Plot

In [8]:
'''Make an output widget for the figure to go in'''
from mpl_toolkits.mplot3d import Axes3D

fig1Output = widgets.Output()

fig1 = plt.figure(figsize=(20,8))

ax1 = fig1.add_subplot(projection='3d')

def updateplot(b):

  ax1.clear()
  for throw in throws:
    if not hasattr(throw, 't'):
      continue

    '''Define x, y, z data'''
    x = throw.pos_g[:,0]*3.28 # Convert to ft
    y = throw.pos_g[:,1]*3.28 # Convert to ft
    z = throw.pos_g[:,2]*3.28 # Convert to ft

    '''Select roll rate as color data for the plot'''
    color = throw.rot_s[:,0]*(180/np.pi) # Convert to deg/s
    colorMax = np.max(abs(color))
    colorMin = -colorMax

    annot = (throw.label + ' \n{:0.0f} ft ').format(throw.getDistance()*3.28)

    ax1.scatter(x, y, z, c=color, cmap='coolwarm', vmin=colorMin, vmax=colorMax, alpha=1, s=40)
    ax1.plot(x, y, '-k', zdir='z', alpha=0.3)
    ax1.text(x[-1],y[-1],0, annot,
             horizontalalignment='right',
             verticalalignment='center',
             bbox=dict(facecolor='white', alpha=0.5))

  ax1.set_xlabel('Forward Travel (ft)', fontsize=14)
  ax1.set_ylabel('Lateral Travel (ft)', fontsize=14)
  ax1.set_zlabel('Elevation (ft)', fontsize=14)
  ax1.set_xlim3d(left=0)
  ax1.set_zlim3d(bottom=0)
  ax1.w_xaxis.set_pane_color((0.3, 0.5, 0.8, 0.15))
  ax1.w_yaxis.set_pane_color((0.3, 0.5, 0.8, 0.15))
  ax1.w_zaxis.set_pane_color((0.3, 0.8, 0.3, 0.15))
  ax1.view_init(elev=30, azim=-90)
  #fig1.tight_layout()
  #fig1.colorbar(flightpath, ax=ax1, label='← Turning / Fading →\nRoll Rate (deg/s)')

  with fig1Output:
    clear_output(wait=True)
    display(ax1.figure)


## Define Flight Data Plot

In [9]:
fig2Output = widgets.Output()

fig2 = plt.figure(figsize=(16,5.5))
ax2 = fig2.gca()
ax2r = ax2.twinx()

def updatedata(b):
  ax2.clear()
  ax2r.clear()
  cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

  for i, throw in enumerate(throws):
    if not hasattr(throw, 't'):
      continue

    xdataSwitch = {
        xdataList[0]: throw.pos_g[:,0]*3.28, # Forward Travel Convert to ft
        xdataList[1]: throw.t # Time
    }

    ydataSwitch = {
        ydataList[0]: [], # No Selection
        ydataList[1]: throw.vel_w[:,0]*2.23694, # Speed convert to mph
        ydataList[2]: throw.lift*101.97, # Lift convert to gram-force
        ydataList[3]: throw.drag*101.97, # Drag convert to gram-force
        ydataList[4]: throw.alpha*(180/np.pi), # Angle of Attack convert to degrees
        ydataList[5]: throw.ori_g[:,0]*(180/np.pi), # Roll Angle convert to degrees
        ydataList[6]: throw.rot_s[:,0]*(180/np.pi), # Roll Rate convert to deg/s
        ydataList[7]: throw.pos_g[:,0]*3.28084, # Forward Travel convert to ft
        ydataList[8]: throw.pos_g[:,1]*3.28084, # Lateral Travel convert to ft
        ydataList[9]: throw.pos_g[:,2]*3.28084 # Elevation convert to ft
    }

    xdataName = xdataInput.value
    ydata1Name = ydataInput1.value
    ydata2Name = ydataInput2.value

    '''Define x, y'''
    x = xdataSwitch.get(xdataName)
    y1 = ydataSwitch.get(ydata1Name)
    y2 = ydataSwitch.get(ydata2Name)

    ax2.plot(x, y1, color = cycle[i], label=throw.label)
    if len(y2):
      ax2r.plot(x, y2, color = cycle[i], linestyle='--')
    else:
      ax2r.axis('off')

  ax2.grid(linestyle='--', color='lightgrey')
  ax2.legend()

  with fig2Output:
    clear_output(wait=True)
    display(ax2.figure)

##Define Disc View Plot

In [13]:
fig3Output = widgets.Output(layout=Layout(height='800px'))

fig3 = plt.figure(figsize=(15,15))
ax3 = fig3.add_subplot(projection = '3d')

def updatedisc(b):
  '''Get throw number from dropdown'''
  num = thrownumInput.value - 1
  throw = throws[num]
  disc = discs[num]

  if not hasattr(throw, 't'):
    ax3.clear()
    ax3.axis('off')
    with fig3Output:
      clear_output(wait=True)
      display(ax3.figure)
    return

  '''Get time from slider'''
  t = disctimeInput.value

  '''Find step nearest to time=t'''
  step = np.argmax(throw.t>=t)

  weight = disc.mass*9.81 # Weight of disc in N. Will be used to normalize all forces on the disc
  ori = throw.ori_g[step]
  alpha = throw.alpha[step]
  beta = throw.beta[step]

  '''Define disc outline'''
  numPoints = 40
  r = np.linspace(0, 2*np.pi, numPoints)
  x_disc = np.cos(r)
  y_disc = np.sin(r)
  z_disc = np.zeros(numPoints)
  discoutline = np.vstack((x_disc, y_disc, z_disc)).T
  p=0
  while p < numPoints:
    discoutline[p] = np.matmul(T_dg(ori), discoutline[p]) # Convert outline from disc coords to ground coords
    p+=1
  discverts = [list(zip(discoutline[:,0],discoutline[:,1],discoutline[:,2]))]

  '''Define lift vector'''
  lift = throw.lift[step]/weight * np.array([0, 0, 1])
  lift = np.matmul(T_ws(alpha), lift)
  lift = np.matmul(T_sd(beta), lift)
  lift = np.matmul(T_dg(ori), lift) # lift vector converted to ground coordinates

  '''Define drag vector'''
  drag = throw.drag[step]/weight * np.array([-1, 0, 0])
  drag = np.matmul(T_ws(alpha), drag)
  drag = np.matmul(T_sd(beta), drag)
  drag = np.matmul(T_dg(ori), drag) # drag vector converted to ground coordinates

  '''Define wind vector pointing towards the disc'''
  maxSpeed = np.max(throw.vel_w[:,0])
  wind = np.array([[2,0,0], [2-throw.vel_w[step,0]/maxSpeed,0,0]])
  p=0
  while p<2:
    wind[p] = np.matmul(T_ws(alpha), wind[p])
    wind[p] = np.matmul(T_sd(beta), wind[p])
    wind[p] = np.matmul(T_dg(ori), wind[p]) # wind vector converted to ground coordinates
    p+=1

  '''Define angle of attack arrow'''
  aoa = np.array([[0.8, 0, 0], [0.8*np.cos(alpha), 0, 0.8*np.sin(alpha)]])
  p=0
  while p<2:
    aoa[p] = np.matmul(T_ws(alpha), aoa[p])
    aoa[p] = np.matmul(T_sd(beta), aoa[p])
    aoa[p] = np.matmul(T_dg(ori), aoa[p])
    p+=1

  '''Define roll angle arrow'''
  roll = np.array([[0, 0.8, 0], [0, 0.8*np.cos(ori[0]), -0.8*np.sin(ori[0])]])

  ax3.clear()
  '''Axis for ground coordinate system'''
  ax3.arrow3D(0,0,0, 1,0,0, mutation_scale=15, arrowstyle="->", linestyle='solid')
  ax3.arrow3D(0,0,0, 0,1,0, mutation_scale=15, arrowstyle="->", linestyle='solid')
  ax3.arrow3D(0,0,0, 0,0,1, mutation_scale=15, arrowstyle="->", linestyle='solid')
  '''Lift vector'''
  ax3.arrow3D(0,0,0,
              lift[0], lift[1], lift[2],
              mutation_scale=20,
              facecolor='lightgreen',
              edgecolor='green')
  '''Drag vector'''
  ax3.arrow3D(0,0,0,
              drag[0], drag[1], drag[2],
              mutation_scale=20,
              facecolor='salmon',
              edgecolor='red')
  '''Gravity vector'''
  ax3.arrow3D(0,0,0,
              0,0,-1,
              mutation_scale=20,
              facecolor='lightgrey',
              edgecolor='dimgrey')
  '''Wind vector'''
  ax3.arrow3D(wind[0,0],wind[0,1],wind[0,2],
              (wind[1,0]-wind[0,0]),(wind[1,1]-wind[0,1]),(wind[1,2]-wind[0,2]),
              mutation_scale=20,
              facecolor='lightblue',
              edgecolor='dodgerblue')
  ax3.plot([0,wind[1,0]], [0,wind[1,1]], [0,wind[1,2]], linestyle=':', color='black')
  '''aoa vector'''
  ax3.arrow3D(aoa[0,0],aoa[0,1],aoa[0,2],
              (aoa[1,0]-aoa[0,0]),(aoa[1,1]-aoa[0,1]),(aoa[1,2]-aoa[0,2]),
              mutation_scale=15,
              arrowstyle='-',
              linestyle='solid',
              color='blue',
              connectionstyle='arc3,rad=0.3')
  '''Disc outlinte'''
  ax3.add_collection3d(Poly3DCollection(discverts, alpha=0.5, facecolor='grey', edgecolor='black'))

  ax3.set_xlim(left=-2, right=2)
  ax3.set_ylim(bottom=-2, top=2)
  ax3.set_zlim(bottom=-2, top=2)

  '''Make a smart legend'''
  liftlabel='Lift = {:.0f} gf'.format(throw.lift[step]*101.97)
  draglabel='Drag = {:.0f} gf'.format(throw.drag[step]*101.97)
  windlabel='Wind = {:.0f} mph'.format(throw.vel_w[step,0]*2.23694)
  gravitylabel='Gravity = {:.0f} gf'.format(disc.mass*9.81*101.97)
  aoalabel = 'AoA = {:.1f}°'.format(throw.alpha[step]*(180/np.pi))
  rolllabel = 'Roll = {:.1f}°'.format(throw.ori_g[step,0]*(180/np.pi))
  ax3.plot([0], [0], '-', zdir='z', color='lightgreen', lw='4', label=liftlabel)
  ax3.plot([0], [0], '-', zdir='z', color='salmon', lw='4', label=draglabel)
  ax3.plot([0], [0], '-', zdir='z', color='lightgrey', lw='4', label=gravitylabel)
  ax3.plot([0], [0], '-', zdir='z', color='lightblue', lw='4', label=windlabel)
  ax3.plot([0], [0], '-', zdir='z', color='blue', label=aoalabel)
  ax3.plot([0], [0], '-', zdir='z', alpha=0, label=rolllabel)
  ax3.legend(loc='center left', fontsize='16', edgecolor='black')
  ax3.axis('off')

  with fig3Output:
    clear_output(wait=True)
    display(ax3.figure)

## Define Widgets and Layout

In [15]:
'''Initialize objects'''
discs = [Disc('Throw 1'), Disc('Throw 2'), Disc('Throw 3')]
throws = [Throw('Throw 1'), Throw('Throw 2'), Throw('Throw 3')]

'''Define input widgets for flight path tab'''
discInput = widgets.Dropdown(options=discList, value=aviar, layout=Layout(width='90%'))
discLabel = widgets.Label('Disc')
massInput = widgets.FloatSlider(value=175,min=130,max=200,readout_format='.0f', layout=Layout(width='90%'))
massLabel = widgets.Label('Mass (g)')
speedInput = widgets.FloatSlider(value=50,min=1,max=100,readout_format='.1f', layout=Layout(width='90%'))
speedLabel = widgets.Label('Speed (mph)')
spinInput = widgets.FloatSlider(value=27,min=1,max=100,readout_format='.1f', layout=Layout(width='90%'))
spinLabel = widgets.Label('Spin (rev/s)')
spindirInput = widgets.Dropdown(options=[('Clockwise',1),('Counter-Clockwise',-1)], value=1, layout=Layout(width='90%'))
blankLabel = widgets.Label(' ')
launchangleInput = widgets.FloatSlider(value=10, min=-10, max=90,readout_format='.1f', layout=Layout(width='90%'))
launchangleLabel = widgets.Label('Launch Angle (deg)')
noseangleInput = widgets.FloatSlider(value=4, min=-10, max=20,readout_format='.1f', layout=Layout(width='90%'))
noseangleLabel = widgets.Label('Nose Angle (deg)')
rollangleInput = widgets.FloatSlider(value=8, min=-180, max=180,readout_format='.1f', layout=Layout(width='90%'))
rollangleLabel = widgets.Label('Roll Angle (deg)')
thrownumLabel = widgets.Label('Select throw number')
thrownumInput = widgets.Dropdown(options=[('Throw 1',1),('Throw 2',2),('Throw 3',3)], value=1, layout=Layout(width='90%'))
throwlabelLabel = widgets.Label('Throw Label')
throwlabelInput = widgets.Text(value='Throw 1', layout=Layout(width='90%'))
huckitInput = widgets.Button(description = 'Huck It!')
huckitInput.style.button_color = 'lightgreen'
clearthrowInput = widgets.Button(description = 'Clear Throw')
clearthrowInput.style.button_color = 'salmon'

'''Link spin to speed'''
def updatespin(change):
  spinInput.value = speedInput.value*0.55

speedInput.observe(updatespin, 'value')

'''Callback function for Huck It button'''
def runSim(b):
  '''Bring in global variables'''
  global throws
  global discs

  '''Get throw number from dropdown'''
  num = thrownumInput.value - 1
  throw = throws[num]
  disc = discs[num]

  '''Collect input values from other widgets'''
  disc.name = discInput.value.name
  disc.aoarange = discInput.value.aoarange
  disc.cl = discInput.value.cl
  disc.cd = discInput.value.cd
  disc.cm = discInput.value.cm
  disc.jxy = discInput.value.jxy
  disc.jz = discInput.value.jz
  disc.diam = discInput.value.diam
  disc.mass = massInput.value / 1000 # Convert to kg
  throw.speed = speedInput.value*0.44704 # Convert to m/s
  throw.spin = spinInput.value*2*np.pi # Convert to rad/s
  throw.spindir = spindirInput.value
  throw.launch_angle = launchangleInput.value*(np.pi/180) # Convert to rad
  throw.nose_angle = noseangleInput.value*(np.pi/180) # Convert to rad
  throw.roll_angle = rollangleInput.value*(np.pi/180) # Convert to rad
  throw.label = throwlabelInput.value

  throws[num] = huckit(disc, throw)
  discs[num] = disc

  updateplot(None)
  updatedata(None)
  updatethrow(None)

'''Callback function for Clear Throw button'''
def clearThrow(b):
  '''Bring in global variables'''
  global throws
  global discs

  '''Get throw number from dropdown'''
  num = thrownumInput.value - 1
  throws[num] = Throw('Throw {:.0f}'.format(num+1))
  discs[num] = Disc('Throw {:.0f}'.format(num+1))

  updateplot(None)
  updatedata(None)
  updatethrow(None)

'''Callback function for Throw dropdown'''
def updatethrow(b):
  '''Get throw number from dropdown'''
  num = thrownumInput.value - 1
  throw = throws[num]
  disc = discs[num]

  '''Check if the throw has been simulated'''
  if hasattr(throw, 't'):
    for d in discList:
      if disc.name==d[0]:
        discInput.value=d[1]
        break
    massInput.value = disc.mass*1000 # Convert to g
    speedInput.value = throw.speed*2.23694 # Convert to mph
    spinInput.value = throw.spin/(2*np.pi) # Convert to rev/s
    spindirInput.value = throw.spindir
    launchangleInput.value = throw.launch_angle/(np.pi/180) # Convert to deg
    noseangleInput.value = throw.nose_angle/(np.pi/180) # Convert to deg
    rollangleInput.value = throw.roll_angle/(np.pi/180) # Convert to deg
    throwlabelInput.value = throw.label
    disctimeInput.max = np.max(throws[num].t) # Update max value of time slider
    if disctimeInput.value > disctimeInput.max:
      disctimeInput.value = disctimeInput.max
  else:
    throwlabelInput.value = throw.name

  updatedisc(None)

'''Throw dropdown Callback'''
thrownumInput.observe(updatethrow, 'value')

'''Button Callbacks'''
huckitInput.on_click(runSim)
clearthrowInput.on_click(clearThrow)

'''Define widget and plot layout'''
inputLayout = widgets.Layout(width='20%')
labelLayout = widgets.Layout(align_items='flex-end')

buttonBox = widgets.HBox([huckitInput,clearthrowInput], layout=Layout(width='90%'))
labelBox = widgets.VBox([discLabel, massLabel, speedLabel, spinLabel, blankLabel,
                         launchangleLabel, noseangleLabel, rollangleLabel, thrownumLabel, throwlabelLabel, blankLabel],
                        layout=labelLayout)
inputBox = widgets.VBox([discInput, massInput, speedInput, spinInput, spindirInput, launchangleInput,
                         noseangleInput, rollangleInput, thrownumInput, throwlabelInput, buttonBox],
                        layout=inputLayout)
plotBox = widgets.VBox([fig1Output], layout=Layout(height='800px', overflow='hidden'))

'''Widgets for flight data tab'''
xdataList = ['Forward Travel (ft)',
             'Time (s)']
ydataList = ['',
             'Speed (m/s)',
              'Lift (gram-force)',
              'Drag (gram-force)',
              'Angle of Attack (deg)',
              'Roll Angle (deg)',
              'Roll Rate (deg/s)',
              'Forward Travel (ft)',
              'Lateral Travel (ft)',
              'Elevation (ft)']

xdataInput = widgets.Dropdown(options=xdataList, value=xdataList[0])
ydataInput1 = widgets.Dropdown(options=ydataList[1:], value=ydataList[4], layout=Layout(width='150px'))
ydataInput2 = widgets.Dropdown(options=ydataList, value=ydataList[0], layout=Layout(width='150px'))
toprow = widgets.HBox([ydataInput1,ydataInput2], layout=Layout(justify_content='space-between', width='100%'))
dataBox = widgets.VBox([toprow, fig2Output, xdataInput], layout=Layout(height='400px', align_items='center'))

'''Callback function for data dropdowns'''
xdataInput.observe(updatedata, 'value')
ydataInput1.observe(updatedata, 'value')
ydataInput2.observe(updatedata, 'value')

'''Widgets for disc view tab'''
disctimeInput = widgets.FloatSlider(value=0, min=0, max=1, description='Flight Time', readout_format='.1f', layout=Layout(height='90%'), orientation='vertical')
discBox = widgets.HBox([disctimeInput, fig3Output], layout=Layout(align_items='center', height='400px', overflow='hidden'))

'''Callback function for time slider'''
disctimeInput.observe(updatedisc, 'value')

tabs = widgets.Tab(children=[plotBox, dataBox, discBox], layout=Layout(width='70%'))
tabs.set_title(0,'Flight Path')
tabs.set_title(1,'Flight Data')
tabs.set_title(2,'Disc View')

mainBox = widgets.HBox([labelBox,inputBox,tabs],layout=Layout(align_items='center'))


#User Interface

In [16]:
display(mainBox)
runSim(None)

<ipython-input-8-12a4515d2688>:41: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax1.w_xaxis.set_pane_color((0.3, 0.5, 0.8, 0.15))
<ipython-input-8-12a4515d2688>:42: MatplotlibDeprecationWarning: The w_yaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use yaxis instead.
  ax1.w_yaxis.set_pane_color((0.3, 0.5, 0.8, 0.15))
<ipython-input-8-12a4515d2688>:43: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax1.w_zaxis.set_pane_color((0.3, 0.8, 0.3, 0.15))


## Input Recommendations




*   ***Mass (grams):*** To simulate throwing a lighter disc, make sure to increase the speed as well. The biggest advantage of throwing a lighter disc is that you can throw it faster. A 10% reduction in weight should result in a ~5% increase in speed.
*   ***Speed (mph):*** A pro disc golfer typically throws around 60 mph for a drive.
*   ***Spin (rev/s):*** Spin is proportional to speed. The spin setting will change automatically as speed changes to maintain a normal ratio between the two. The spin can still be changed manually to evaluate unique scenarios.
*   ***Rotation:*** Clockwise is for right hand backhand and left hand forehand throws. Counter-clockwise is for right hand forehand and left hand backhand throws.
*   ***Launch Angle (deg):*** Launch angle is the direction of the throw relative to horizontal. Positive is up.
*   ***Nose Angle (deg):*** Nose angle is the angle of the disc relative to the throwing line. If the launch angle is 5° and the nose angle is 5°, the total angle of the disc to horizontal ground will be 10°. Select the angle of attack in Flight Data to see how the effective nose angle changes throughout the flight, even though the disc maintains the same pitch relative to the ground. You can also see the angle of attack change in the Disc View, as the oncoming wind moves further under the disc while it is descending.
*   ***Roll Angle (deg):*** Roll angle is the tilt of the disc to the left or right. Positive is a hyzer angle. Negative is anhyzer.

##Disc Flight Physics

In a nutshell...

There are three main aerodynamic forces acting on a disc in flight; lift, drag and pitching moment. Lift is what keeps the disc in the air instead of dropping like a rock. Drag slows the disc down. Pitching moment wants to tilt the disc forward or backward and results from an imbalance in the lift force. If there is more lift on the rear of the disc than on the front, the disc will want to pitch forward. However, since the disc is spinning, a pitching moment will actually cause the disc to roll to the right or left. Neat! At low nose angles (disc is flat) a right hand backhand throw will want to turn to the right. As the disc slows down and starts falling, the pitching moment will eventually reverse and cause the disc to fade (turn to the left). The result is the charactistic s-shaped flight path of a golf disc.